En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Llamada del archivo para su procesamiento

In [1]:
file_path = r"D:\\Personal\\Prueba_Tecnica\\LATAM\\farmers-protest-tweets-2021-2-4.json"

In [2]:
#pip install py-spy
#pip install pyspy
#pip install memory-profiler
#pip install emoji

In [2]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import cProfile
from typing import List, Tuple
from datetime import datetime
from memory_profiler import memory_usage
import emoji


## Punto 1 datos temporales

### **Función inicial**

**Consideraciones con las que se creó la función**

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas.

2. Se agrega una nueva columna al DataFrame para almacenar las fechas en formato date.

3. La columna de usuario se normaliza para extraer los datos pertinentes, ya que contiene información sobre los usuarios que realizaron las interacciones.

4. Para reducir el tamaño del DataFrame y optimizar el procesamiento, se crea un DataFrame más pequeño que incluye solo las columnas esenciales, como el ID de la interacción y la fecha, junto con los datos del usuario en columnas separadas.

5. Se utiliza groupby para crear una tabla que contenga las fechas, los usuarios y el recuento de interacciones por usuario en cada fecha.

6. Se crea una segunda tabla que calcula el total de interacciones por fecha, lo que permite ordenar la tabla por la fecha de manera efectiva.

7. Se uner la tabla del punto 5 y 6 y e ordena la tabla resultante de manera que las fechas con mayor actividad aparezcan primero, y dentro de cada fecha, los usuarios más activos se encuentren en la parte superior.

8. Se selecciona el primer registro de cada fecha, lo que proporciona el nombre de usuario con la mayor cantidad de interacciones para esa fecha en particular.

9. Finalmente, se crea una lista de tuplas que contiene las fechas y los nombres de usuario correspondientes, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.

In [4]:
def q1_return_inicial(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    df = pd.read_json(file_path, lines=True)
    
    # Modificar la columna de fechas
    df['date_tw'] = pd.to_datetime(df['date']).dt.date
    
    # Normalizar la columna 'user' para obtener datos de usuarios
    df_user = json_normalize(df['user'])
    
    # Unir el DataFrame original con los datos normalizados de usuarios
    df = pd.concat([df[['id','date_tw']], df_user.add_prefix('user.')], axis=1)
    
    # Agrupar por fecha y usuario, contar las interacciones y obtener el total de interacciones por fecha
    df_tw_u = df.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')
    
    # Unir los DataFrames y ordenar por conteo total y por usuario
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw') \
                .sort_values(by=['count_total', 'count_user'], ascending=[False, False])
    
    # Seleccionar el usuario con más interacciones para cada fecha
    df_tw_u_f = df_tw_u.groupby('date_tw').first().reset_index().sort_values(by=['count_total'], ascending=False)
    
    # Seleccionar las primeras 10 filas y crear una lista de tuplas de fecha y usuario
    lista_resultado = list(zip(df_tw_u_f['date_tw'], df_tw_u_f['user.username']))[:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [5]:
q1_return_inicial(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

**Evaluación del tiempo de ejecución**

In [6]:
cProfile.run('q1_return_inicial(file_path)')

         6962513 function calls (6844681 primitive calls) in 13.221 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.420    0.420   12.998   12.998 3155568970.py:1(q1_return_inicial)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(allclose)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
       29    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
       35    

**Memoria total utilizada**

El ejecutar la función memory_usage((q1_return_inicial, (file_path,))), se obtiene la memoria usada durante la ejecución de la función q1_return_inicial, así que para obtener el valor total de memoria utilizada se realiza la suma de los valores de la lista. 

In [7]:
np.sum(memory_usage((q1_return_inicial, (file_path,))))

197608.4765625

### **Intentos de optimización de la función inicial**

#### **Función 1**

Cambios:

1. En lugar de cargar todo el archivo JSON de una vez, se lee línea por línea para evitar cargar todo el archivo en memoria al mismo tiempo, lo que reduce el uso de memoria.

2. En lugar de usar add_prefix, se asignan los nombres de columnas directamente para mayor claridad.

3. El filtro para encontrar el usuario con mayor interaccion por fecha no se hace por método first(), si no que se usa encontrando los índices de estos usuarios y luego filtrando la tabla que contiene la información de fechas nombre de usarios, en esta misma tabla se filtran los 10 registros que pide el ejecicio.

4. Se crea un ciclo para la creación de la lista de datos requeridos. 

In [8]:
def q1_return_v1(file_path: str) -> List[Tuple[datetime.date, str]] :
    # Punto 1: Lista para almacenar los datos JSON
    data = []

    # Punto 2: Conjunto para almacenar los nombres de las claves únicas
    unique_keys = set()

    # Punto 3: Abrir el archivo JSON y leer línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Punto 4: Cargar la línea como un objeto JSON
            json_data = json.loads(line.strip())
            
            # Punto 5: Agregar el objeto JSON a la lista
            data.append(json_data)
            
            # Punto 6: Actualizar el conjunto de claves únicas
            unique_keys.update(json_data.keys())

    # Punto 7: Crear DataFrame con los datos JSON
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    # Punto 8: Modificar la columna de fechas para obtener el formato AAAA-MM-DD
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date
    
    # Punto 9: Crear un dataframe con los datos expandidos de los usuarios que luego se une al df inicial
    df_user = json_normalize(df_inicial['user'])
    ## Se asigna el prefijo a los nombres de las columnas para no perder el origen
    df_user.columns = ['user.'+col for col in df_user.columns ]
    ## Se une el df de usuarios el df inicial y se eliminan los ínidices
    df_inicial = pd.concat([df_inicial[['id','date_tw']].copy(),df_user],axis=1).reset_index(drop=True)   
    
    # Punto 10: Se crean groupby para procesar solo la información de fechas y nombres de usuario, agregando la columna del conteo de interacciones de usuarios
    df_tw_u = df_inicial.groupby(['date_tw','user.username'])['id'].count().reset_index(name='count_user')
    
    # Punto 11: Se crea un df nuevo que contiene solo el total de interacciones por fecha.
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().sort_values(ascending=False).reset_index(name='count_total')
    ## Se unen el df original de fechas, usuarios, conteo de interacciones por usuario al df que contiene las fechas e interacciones por fecha, se ordena de tal forma que el conteo total y por usuario es descendente
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total','count_user'],ascending=[False,False])
    ## Se buscan los índices de la primera fila de cada fecha.
    indeices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    ## Se filtra del df de twits de tal forma que se ordene de la fecha de mayor interaccion a la menor, solo los primeros 10 registros
    df_tw_u_f = df_tw_u.loc[indeices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10,::].reset_index(drop=True)
    
    # Punto 12: Se crea un ciclo para llenar una lista con la tuplas de fecha y el usuario con mayor cantidad de interacciones en X para ese día 
    lista_resultado = []
    for i in range(len(df_tw_u_f)):
        lista_resultado.append((df_tw_u_f['date_tw'][i],df_tw_u_f['user.username'][i]))
    
    return lista_resultado


**Resultado de ejecución del a función**

In [9]:
q1_return_v1(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

**Evaluación del tiempo de ejecución**

In [10]:
cProfile.run('q1_return_v1(file_path)')

         8457475 function calls (8339785 primitive calls) in 10.666 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.968    0.968   10.037   10.037 2760368282.py:1(q1_return_v1)
        1    0.000    0.000    0.000    0.000 2760368282.py:29(<listcomp>)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(allclose)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
       34    0.000    0.000    0.126

**Memoria total utilizada**

In [11]:
np.sum(memory_usage((q1_return_v1, (file_path,))))

77776.5390625

#### **Función 2**

Cambios:

Si bien se mantiene una estructura muy similar a la de la función 1, se realizan algunos pequeños cambios:

1. Al crear el concat entre el DataFrame inicila y el DataFrame de usuarios, en el segundo caso se usa solo la columna objetivo, que es el nombre de usuario.
2. La lista de resultados se realiza con una lista de compresión en iteración sobre las líneas del df final.

In [43]:
def q1_return_t2(file_path: str) -> List[Tuple[datetime.date, str]]:
    data = []
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date

    df_user = json_normalize(df_inicial['user'])
    df_user.columns = ['user.'+col for col in df_user.columns]
    df_inicial = pd.concat([df_inicial[['id','date_tw']], df_user['user.username']], axis=1).reset_index(drop=True)

    df_tw_u = df_inicial.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')

    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total', 'count_user'], ascending=[False, False])

    indices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    df_tw_u_f = df_tw_u.loc[indices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10].reset_index(drop=True)

    lista_resultado = [(row['date_tw'], row['user.username']) for _, row in df_tw_u_f.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [44]:
q1_return_t2(file_path)

**Evaluación del tiempo de ejecución**

In [14]:
cProfile.run('q1_return_t2(file_path)')

         8457048 function calls (8339342 primitive calls) in 10.196 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.927    0.927    9.534    9.534 365856321.py:1(q1_return_t2)
        1    0.000    0.000    0.000    0.000 365856321.py:15(<listcomp>)
        1    0.000    0.000    0.002    0.002 365856321.py:26(<listcomp>)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(allclose)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        9    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        9    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        4    0.000    0.000    0.000    0.000 <__array_

**Memoria total utilizada**

In [15]:
np.sum(memory_usage((q1_return_t2, (file_path,))))

77078.71875

## Punto 2 emojies

In [16]:
def q2_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)
        
    df_emojies = pd.DataFrame(lista_emojies,columns=['emojie'])
    
    df_ef = pd.DataFrame(df_emojies.value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [17]:
q2_run_inicial(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

**Evaluación del tiempo de ejecución**

In [18]:
cProfile.run('q2_run_inicial(file_path)')

         2338463 function calls (2338339 primitive calls) in 8.680 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.889    0.889    8.136    8.136 2406996805.py:1(q2_run_inicial)
   117407    0.038    0.000    1.236    0.000 2406996805.py:17(extraer_emojis)
   117407    1.198    0.000    1.198    0.000 2406996805.py:18(<listcomp>)
        1    0.000    0.000    0.001    0.001 2406996805.py:36(<listcomp>)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        9    0.000    0.000    0.026    0.003 <__array_fun

**Memoria total utilizada**

In [19]:
np.sum(memory_usage((q2_run_inicial, (file_path,))))

73019.28125

### **Intentos de optimización de la función inicial**

### **Función 1**

In [20]:
def q2_run_v1(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)

    df_ef = pd.DataFrame(pd.DataFrame(lista_emojies,columns=['emojie']).value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


In [21]:
q2_run_v1(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

In [22]:
cProfile.run('q2_run_v1(file_path)')

         2338463 function calls (2338339 primitive calls) in 9.062 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.925    0.925    8.561    8.561 3950940001.py:1(q2_run_v1)
   117407    0.033    0.000    1.625    0.000 3950940001.py:17(extraer_emojis)
   117407    1.592    0.000    1.592    0.000 3950940001.py:18(<listcomp>)
        1    0.000    0.000    0.001    0.001 3950940001.py:34(<listcomp>)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        9    0.000    0.000    0.024    0.003 <__array_function

In [23]:
np.sum(memory_usage((q2_run_v1, (file_path,))))

74121.0859375

In [24]:
from collections import Counter

def q2_run_v2(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    # Crear una lista plana con todos los emojis encontrados en todas las filas
    lista_emojies = [emoji for fila in df_inicial['emojis'] for emoji in fila]
    
    # Crear DataFrame con los emojis y su frecuencia
    df_ef = pd.DataFrame(Counter(lista_emojies).most_common(), columns=['emojie', 'Cuenta'])
        
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado

In [25]:
q2_run_v2(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

In [26]:
cProfile.run('q2_run_v2(file_path)')

         2219117 function calls (2219026 primitive calls) in 9.171 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    0.040    0.000    1.233    0.000 2893459868.py:19(extraer_emojis)
   117407    1.192    0.000    1.192    0.000 2893459868.py:20(<listcomp>)
        1    0.011    0.011    0.011    0.011 2893459868.py:26(<listcomp>)
        1    0.914    0.914    8.532    8.532 2893459868.py:3(q2_run_v2)
        1    0.000    0.000    0.001    0.001 2893459868.py:34(<listcomp>)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        6    0.000    0.000    0.031    0.005 <__array_function__ internals>:177(c

In [27]:
np.sum(memory_usage((q2_run_v2, (file_path,))))

71865.89453125

## Punto 3 influyentes

### **Encontrar donde se mencionan a los usuarios**

Se evidencia que tanto en la columna content como en la columna mentionedUsers hat mención a los usuarios, por tanto, es necesario realizar la exploración aleatoria de si los usuarios mencionados en content con los mismo de mentionedUsers

In [31]:
def qinicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    return df_inicial

resultado = qinicial(file_path)

In [41]:
resultado['mentionedUsers'] = resultado['mentionedUsers'].explode()
pd.set_option('display.max_colwidth', 200) 
resultado[~resultado['mentionedUsers'].isnull()][['content','mentionedUsers']]

C:\Users\Sebastian\AppData\Local\Temp\ipykernel_9936\1380774437.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  resultado['mentionedUsers'] = resultado['mentionedUsers'].explode()


ValueError: cannot reindex on an axis with duplicate labels

In [3]:
def q3_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)
        
    df_emojies = pd.DataFrame(lista_emojies,columns=['emojie'])
    
    df_ef = pd.DataFrame(df_emojies.value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return df_inicial



In [4]:
resultado3 = q3_run_inicial(file_path)


In [5]:
resultado3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117407 entries, 0 to 117406
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               117407 non-null  int64 
 1   replyCount       117407 non-null  int64 
 2   retweetCount     117407 non-null  int64 
 3   content          117407 non-null  object
 4   quoteCount       117407 non-null  int64 
 5   sourceUrl        116495 non-null  object
 6   url              117407 non-null  object
 7   sourceLabel      116495 non-null  object
 8   retweetedTweet   0 non-null       object
 9   date             117407 non-null  object
 10  lang             117407 non-null  object
 11  likeCount        117407 non-null  int64 
 12  quotedTweet      41436 non-null   object
 13  outlinks         117407 non-null  object
 14  tcooutlinks      117407 non-null  object
 15  conversationId   117407 non-null  int64 
 16  source           117407 non-null  object
 17  mentionedU

In [39]:
resultado3['mentionedUsers'][117392]

[{'username': 'BeingSalmanKhan',
  'displayname': 'Salman Khan',
  'id': 132385468,
  'description': None,
  'rawDescription': None,
  'descriptionUrls': None,
  'verified': None,
  'created': None,
  'followersCount': None,
  'friendsCount': None,
  'statusesCount': None,
  'favouritesCount': None,
  'listedCount': None,
  'mediaCount': None,
  'location': None,
  'protected': None,
  'linkUrl': None,
  'linkTcourl': None,
  'profileImageUrl': None,
  'profileBannerUrl': None,
  'url': 'https://twitter.com/BeingSalmanKhan'}]

,username,displayname,id,description,rawDescription,descriptionUrls,verified,created,followersCount,friendsCount,...,favouritesCount,listedCount,mediaCount,location,protected,linkUrl,linkTcourl,profileImageUrl,profileBannerUrl,url
0,narendramodi,Narendra Modi,1.883978e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://twitter.com/narendramodi
1,DelhiPolice,#DilKiPolice Delhi Police,1.850705e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://twitter.com/DelhiPolice
2,Kisanektamorcha,Kisan Ekta Morcha,1.338537e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://twitter.com/Kisanektamorcha
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ReallySwara,Swara Bhasker,1.546102e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://twitter.com/ReallySwara
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
resultado3['content'][117392]

'@BeingSalmanKhan Salmon Khan, wear bangles and chop off your ****. #SpinelessCelebs #AntiNationalBollywood #FarmersProtest #BJPAgainstFarmers'

Se observa que los @ de content son los mismos que hay en la columna mentionedusers por lo que será esa columna la que se use

In [ ]:
json_normalize(resultado3['mentionedUsers'].explode())

## Código no optimizado

In [28]:
fgdsghg

# Lista para almacenar los DataFrames individuales
data = []
lista_keys = []

# Abrir el archivo y leer línea por línea
with open(file_path, 'r') as file:
    for line in file:
        # Crear un diccionariodataframe con una sola columna "json" para cada línea
        diccionario_df = pd.DataFrame({'json': [line.strip()]})
        # Agregar el DataFrame a la lista
        data.append(diccionario_df)
        lista_keys.extend(list(diccionario_df['json'].apply(json.loads)[0].keys()))

NameError: name 'fgdsghg' is not defined

In [ ]:
# Concatenar todos los encabezados únicos para luego formar un dataframe genetal
lista_keys_unicas = set(lista_keys)
#Crear data frame vacío para luego concatenar
df_inicial = pd.DataFrame(columns=lista_keys_unicas)
#Iterar para concatenar todos los df
for df in data: 
    df= df.reindex(columns=lista_keys_unicas)
    df_inicial= pd.concat([df_inicial,df])